In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
#test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
book_train = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet')
trade_train = pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet')
#book_test = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet')
#print(train.head(2))
#print(test.head(2))
#print(book_train.head(2))
#print(trade_train.head(2))
#print(trade_test.head(2))

In [ ]:
print(train.head(2))

In [ ]:
print(book_train.head(2))

In [ ]:
print(trade_train.head(2))

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [ ]:
# To calculate WAP and realized volatility per stock id and time id
def realized_volatility_per_time_id(file_path, prediction_column_name):
    #print(file_path)
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [ ]:
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized

In [ ]:
import os
import glob
list_order_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*')
df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_name='pred')

In [ ]:
list_order_book_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')
df_past_realized_test = past_realized_volatility_per_stock(list_file=list_order_book_file_test,
                                                           prediction_column_name='test_past_vol')

In [ ]:
df_past_realized_train.head()

In [ ]:
df_past_realized_test.head()

In [ ]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
#train = train[['row_id','target']]
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')
df_joined.columns = ['stock_id', 'time_id', 'target', 'row_id', 'past_vol']
#df_joined.drop('row_id', axis=1, inplace=True)
#print(train.head())

In [ ]:
df_joined.head()

In [ ]:
def trade_book_total(file_path, prediction_column_name):
    #print(file_path)
    df_trade_data = pd.read_parquet(file_path)
    df_trade_data['tot_price'] = df_trade_data['price'] * df_trade_data['size']
    df_total_trade_data =  pd.DataFrame(df_trade_data.groupby(['time_id']).sum('tot_price')).reset_index()
    df_total_trade_data = df_total_trade_data.rename(columns = {'tot_price':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_total_trade_data['row_id'] = df_total_trade_data['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_total_trade_data [['row_id',prediction_column_name]]

In [ ]:
def trade_book_total_value(list_file,prediction_column_name):
    df_trade_book = pd.DataFrame()
    for file in list_file:
        #print(file)
        df_trade_book = pd.concat([df_trade_book,trade_book_total(file, prediction_column_name)])
    return df_trade_book

In [ ]:
import os
import glob
list_trade_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/*')
df_past_realized_train_trade = trade_book_total_value(list_file=list_trade_book_file_train,
                                                           prediction_column_name='total_val')

In [ ]:
list_trade_book_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/*')
df_past_realized_test_trade = trade_book_total_value(list_file=list_trade_book_file_test,
                                                           prediction_column_name='total_val')

In [ ]:
df_past_realized_train_trade.head()

In [ ]:
df_past_realized_test_trade.head()

In [ ]:
###train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
#train = train[['row_id','target']]
df_joined_new = df_joined.merge(df_past_realized_train_trade, on = ['row_id'], how = 'left')
df_joined_new.columns = ['stock_id', 'time_id', 'target', 'row_id','past_vol','past_trade_tot']
df_joined_new.drop('row_id', axis=1, inplace=True)
#print(train.head())

In [ ]:
df_joined_new.head()

In [ ]:
df_joined_new.isnull().sum()

In [ ]:
df_joined_new.fillna(0, inplace=True)

In [ ]:
df_joined_new.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
np.random.seed(0) # to select same sample everytime
df_train, df_test = train_test_split(df_joined_new, train_size=0.7, test_size=0.3, random_state=42)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
#num_vars = ['stock_id', 'time_id', 'target', 'past_vol']
#df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

In [ ]:
## Dividing into X & y sets for model building

y_train = df_train.pop('target')
X_train = df_train

In [ ]:
X_train.isnull().sum()

In [ ]:
y_train.isnull().sum()

In [ ]:
## Fitting the model

lm = LinearRegression()
lm.fit(X_train, y_train)
#print(X_train.info())
#print(y_train.info())

In [ ]:
rfe = RFE(lm, 4)
rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_, rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]
col

In [ ]:
X_train_rfe = X_train[col]

In [ ]:
vif = pd.DataFrame()
X = X_train_rfe
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_rfe

In [ ]:
## First model

# Adding a constant variable 
X_train_lm = sm.add_constant(X_train_rfe)

## Running the model
lm1 = sm.OLS(y_train, X_train_lm).fit()

In [ ]:
lm1.summary()

In [ ]:
y_train_pred = lm1.predict(X_train_lm)

In [ ]:
X_train_lm

In [ ]:
y_train_pred

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)

In [ ]:
#num_vars = ['stock_id', 'time_id', 'target', 'past_vol']
#df_test[num_vars] = scaler.fit_transform(df_test[num_vars])

In [ ]:
y_test = df_test.pop('target')
X_test = df_test

In [ ]:
X_test.head()

In [ ]:
X_test.isnull().sum()

In [ ]:
# Now let's use our model to make predictions.

# Creating X_test_new dataframe by dropping variables from X_test
#X_test_new = X_test[X_train.columns]

# Adding a constant variable 
X_test_new = sm.add_constant(X_test)

In [ ]:
X_test_new.head()

In [ ]:
X_test_new.isnull().sum()

In [ ]:
y_test.isnull().sum()

In [ ]:
y_test_pred = lm1.predict(X_test_new)

In [ ]:
y_test_pred.isnull().sum()

In [ ]:
# Plotting y_test and y_pred to understand the spread.
import matplotlib.pyplot as plt
fig = plt.figure()
plt.scatter(y_test,y_test_pred)
fig.suptitle('y_test vs y_test_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_test_pred', fontsize=16)   

In [ ]:
r2_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
book_test = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_test.parquet')
print(book_test)

In [ ]:
trade_test = pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_test.parquet')
trade_test

In [ ]:
print(df_past_realized_test)

In [ ]:
test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
test

In [ ]:
df_past_realized_test_trade.head()

In [ ]:
df_past_realized_test.head()

In [ ]:
test.head(2)

In [ ]:
#test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
df_test_joined = test.merge(df_past_realized_test[['row_id','test_past_vol']], on = ['row_id'], how = 'left')
df_test_joined.columns = ['stock_id', 'time_id', 'row_id', 'past_vol']
#df_test_joined.drop('row_id', axis=1, inplace=True)
print(df_test_joined.head())

In [ ]:
###train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
#train = train[['row_id','target']]
df_test_joined_new = df_test_joined.merge(df_past_realized_test_trade, on = ['row_id'], how = 'left')
df_test_joined_new.columns = ['stock_id', 'time_id', 'row_id','past_vol','test_past_vol']
df_test_joined_new.drop('row_id', axis=1, inplace=True)
#print(train.head())

In [ ]:
#num_vars = ['stock_id', 'time_id', 'test_past_vol']
#df_test_joined[num_vars] = scaler.fit_transform(df_test_joined[num_vars])
df_test_joined_new

In [ ]:
X_test_joined = df_test_joined_new

In [ ]:
X_test_joined = sm.add_constant(df_test_joined_new)
#print(df_test_joined.shape)
print(X_test_joined)

In [ ]:
X_test_joined.dropna(axis=0, inplace=True)
X_test_joined

In [ ]:
#X_test_joined = sm.add_constant(X_test_joined)
y_test_joined_pred = lm1.predict(X_test_joined)

In [ ]:
y_test_joined_pred

In [ ]:
results = pd.concat([df_test_joined_new, y_test_joined_pred], axis=1)
results['row_id'] = results['stock_id'].astype(str) + '-' + results['time_id'].astype(str)
results.drop(['stock_id', 'time_id', 'past_vol', 'test_past_vol'], axis=1, inplace=True)
results.columns = ['target', 'row_id']
results = results[['row_id', 'target']]
results_final = results.fillna(0)
results_final

In [ ]:
results_final.to_csv('submission.csv',index = False)